## Predicting Credit Card Defaults

This notebook compares four common classification algorithms individually and in ensemble for their accuracy in predicting credit card defaults using ROC as a selection metric. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import GammaRegressor
from sklearn.neural_network import MLPClassifier

from sklearn import metrics as metrics


In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
ccdefaults = pd.read_parquet("CCDefaults.parquet")

In [ ]:
#ccdefaults.var().sort_values()

In [ ]:
y = ccdefaults["default_oct"]
other_targets = ccdefaults[["avg_bill_3", "percentile_bin"]]

In [ ]:
ccdefaults.groupby("percentile_bin").median()["avg_bill"]

In [ ]:
X = ccdefaults.drop(columns = ["default_oct", "avg_bill_3", "percentile_bin"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
y_train.value_counts()

In [ ]:
dummy_clf = DummyClassifier(strategy = "stratified")

dummy_clf.fit(X_train, y_train)

dummy_pr = dummy_clf.predict(X_test)

In [ ]:
def get_metrics(true, pred, name):
    scores = {"Model":[], "ROC_AUC": [], "F1": [], "Brier": []}
    scores["Model"].append(name)
    scores["ROC_AUC"].append(metrics.roc_auc_score(true, pred))
    scores["F1"].append(metrics.f1_score(true, pred))
    #scores["Brier"].append(metrics.brier_score_loss(true, pred))
    return pd.DataFrame(scores)
    

In [ ]:
scores = pd.DataFrame(get_metrics(y_test, dummy_pr, "Dummy Classifier"))

In [ ]:
lr = LogisticRegression().fit(X_train_scaled, y_train)
lr_pred = lr.predict(X_test_scaled)

scores = scores.append(get_metrics(y_test, lr_pred, "Logit"), ignore_index= True)

In [ ]:
lambdas = list(np.power(10.0, np.arange(-4, 4)))

lr_lasso = LogisticRegressionCV(penalty='l1'
                                , Cs = lambdas
                                , solver = 'liblinear'
                                , cv = 5).fit(X_train_scaled, y_train)

lr_lasso_pred = lr_lasso.predict(X_test_scaled)
scores = scores.append(get_metrics(y_test, lr_lasso_pred, "Logit Lasso"), ignore_index= True)

In [ ]:
lr_lasso.C_

In [ ]:
lasso_coefs = pd.DataFrame(lr_lasso.coef_, columns=X_train.columns)
lasso_coefs[lasso_coefs != 0].dropna(axis = "columns").transpose()

In [ ]:
rf_grid = { "n_estimators": np.arange(15, 60, step = 15)
          , "max_depth": np.arange(1, 20)}

rf = RandomForestClassifier()
rf_gs = GridSearchCV(rf, rf_grid, cv = 5)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
rf_pred = rf_gs.best_estimator_.predict(X_test)

scores = scores.append(get_metrics(y_test, rf_pred, "Random Forest"), ignore_index= True)

In [ ]:
#X_train_scaled.shape[0]/(3*(X_train_scaled.shape[1]+1))

In [ ]:
nnet = keras.Sequential([
    keras.layers.Dense(180, activation = "relu", input_shape = (X_train_scaled.shape[1],)),
    keras.layers.Dense(180, activation = "relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(360, activation = "relu"),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(1, activation = "sigmoid"),
]
)

nnet.summary()

metrics = [
    keras.metrics.AUC(name="AUC"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall")
]

callbacks = [keras.callbacks.ModelCheckpoint("ccdefaults_checkpoint_{epoch}.h5")]

nnet.compile(optimizer=keras.optimizers.Adam(0.001),
             loss="binary_crossentropy",
             metrics=metrics)

In [ ]:
epochs = nnet.fit(X_train_scaled,
         y_train,
         batch_size=2048,
         epochs = 30,
         validation_data=(X_test_scaled, y_test),
         verbose=2)

In [ ]:
#plt.plot(epochs.history["val_AUC"])
#plt.show()

In [ ]:
nnet_eval = nnet.evaluate(X_test_scaled, y_test)

In [ ]:
scores = scores.append(
    pd.DataFrame(
        {"Model":["Neural Network"], 
         "ROC_AUC": [nnet_eval[1]],
         "F1": [2*(nnet_eval[6]*nnet_eval[7])/(nnet_eval[6]+nnet_eval[7])],
         "Balanced Accuracy": [round(nnet_eval[1], 4)]]}), ignore_index= True)

#"Brier": [((nnet_pred - y_test.to_numpy())**2).mean()

In [ ]:
scores

It looks like the Neural Network approach is the most accurate, but who cares? Find out in the enrapturing conclusion to the CCDefaults saga in:

[Credit Card Defaults 2: Tokyo Drift!](CCDefaults_FinancialProjections.ipynb)

### References

Yeh, I. C., & Lien, C. H. (2009). The comparisons of data mining techniques for the predictive accuracy of probability of default of credit card clients. Expert Systems with Applications, 36(2), 2473-2480.